In [0]:
%run /Workspace/Users/pragatitak007@gmail.com/configs

/databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: Could not open file </Workspace/Users/pragatitak007@gmail.com/f1_project_notebook_bronze_ingestion> for safe execution.
  warn('Could not open file <%s> for safe execution.' % fname)


In [0]:
#delta_path = "abfss://containerf1project@adfstorageaccount3.dfs.core.windows.net/bronze/raw_data/tables"
#silver_path = "abfss://containerf1project@adfstorageaccount3.dfs.core.windows.net/silver/tables"

In [0]:
from pyspark.sql.functions import current_timestamp

circuits_df_silver = spark.read.format("delta").load(delta_path + "/circuits")
#circuits_df_silver.show(5)

circuits_df_silver_transformed = circuits_df_silver.withColumnRenamed("circuitId","circuit_id") \
    .withColumnRenamed("circuitRef","circuit_ref") \
        .withColumnRenamed("lat","latitude") \
        .withColumnRenamed("lng","longitude") \
            .withColumnRenamed("alt","altitude") \
                .withColumn("ingestion_date", current_timestamp()) \
                    .drop("url")


circuits_df_silver_transformed.show()

circuits_df_silver_transformed.write.format("parquet").mode("overwrite").save(silver_path + "/circuits")

+----------+--------------+--------------------+------------+---------+--------+---------+--------+--------------------+
|circuit_id|   circuit_ref|                name|    location|  country|latitude|longitude|altitude|      ingestion_date|
+----------+--------------+--------------------+------------+---------+--------+---------+--------+--------------------+
|         1|   albert_park|Albert Park Grand...|   Melbourne|Australia|-37.8497|  144.968|      10|2025-03-06 09:34:...|
|         2|        sepang|Sepang Internatio...|Kuala Lumpur| Malaysia| 2.76083|  101.738|      18|2025-03-06 09:34:...|
|         3|       bahrain|Bahrain Internati...|      Sakhir|  Bahrain| 26.0325|  50.5106|       7|2025-03-06 09:34:...|
|         4|     catalunya|Circuit de Barcel...|    Montmeló|    Spain|   41.57|  2.26111|     109|2025-03-06 09:34:...|
|         5|      istanbul|       Istanbul Park|    Istanbul|   Turkey| 40.9517|   29.405|     130|2025-03-06 09:34:...|
|         6|        monaco|   Ci

In [0]:
from pyspark.sql.functions import concat_ws, to_timestamp, col, concat, lit

races_df_silver = spark.read.format("delta").load(delta_path + "/races")

races_df_silver_transformed = races_df_silver.withColumnRenamed("raceId","race_id") \
       .withColumnRenamed("year","race_year") \
            .withColumnRenamed("circuitId","circuit_id") \
                .withColumn("race_timestamp", to_timestamp(concat_ws(" ", races_df_silver.date, races_df_silver.time), "yyyy-MM-dd HH:mm:ss")) \
                     .withColumn("ingestion_date", current_timestamp()) \
                         .drop("url")

races_df_silver_transformed.show()

races_df_silver_transformed.write.format("parquet").mode("overwrite").save(silver_path + "/races")
           

+-------+---------+-----+----------+--------------------+----------+--------+-------------------+--------------------+
|race_id|race_year|round|circuit_id|                name|      date|    time|     race_timestamp|      ingestion_date|
+-------+---------+-----+----------+--------------------+----------+--------+-------------------+--------------------+
|      1|     2009|    1|         1|Australian Grand ...|2009-03-29|06:00:00|2009-03-29 06:00:00|2025-03-06 09:40:...|
|      2|     2009|    2|         2|Malaysian Grand Prix|2009-04-05|09:00:00|2009-04-05 09:00:00|2025-03-06 09:40:...|
|      3|     2009|    3|        17|  Chinese Grand Prix|2009-04-19|07:00:00|2009-04-19 07:00:00|2025-03-06 09:40:...|
|      4|     2009|    4|         3|  Bahrain Grand Prix|2009-04-26|12:00:00|2009-04-26 12:00:00|2025-03-06 09:40:...|
|      5|     2009|    5|         4|  Spanish Grand Prix|2009-05-10|12:00:00|2009-05-10 12:00:00|2025-03-06 09:40:...|
|      6|     2009|    6|         6|   Monaco Gr

In [0]:
races_df_silver = spark.read.format("delta").load(delta_path + "/races")

races_df_silver_transformed = races_df_silver.withColumnRenamed("raceId","race_id") \
           .withColumnRenamed("year","race_year") \
                .withColumnRenamed("circuitId","circuit_id") \
                     .withColumn("race_timestamp", to_timestamp(concat_ws(" ", races_df_silver.date, races_df_silver.time), "yyyy-MM-dd HH:mm:ss")) \
                          .withColumn("ingestion_date", current_timestamp()) \
                              .drop("url")

races_df_silver_transformed.show()

races_df_silver_transformed.write.partitionBy("race_year").format("parquet").mode("overwrite").save(silver_path + "/races")

+-------+---------+-----+----------+--------------------+----------+--------+-------------------+--------------------+
|race_id|race_year|round|circuit_id|                name|      date|    time|     race_timestamp|      ingestion_date|
+-------+---------+-----+----------+--------------------+----------+--------+-------------------+--------------------+
|      1|     2009|    1|         1|Australian Grand ...|2009-03-29|06:00:00|2009-03-29 06:00:00|2025-03-06 09:35:...|
|      2|     2009|    2|         2|Malaysian Grand Prix|2009-04-05|09:00:00|2009-04-05 09:00:00|2025-03-06 09:35:...|
|      3|     2009|    3|        17|  Chinese Grand Prix|2009-04-19|07:00:00|2009-04-19 07:00:00|2025-03-06 09:35:...|
|      4|     2009|    4|         3|  Bahrain Grand Prix|2009-04-26|12:00:00|2009-04-26 12:00:00|2025-03-06 09:35:...|
|      5|     2009|    5|         4|  Spanish Grand Prix|2009-05-10|12:00:00|2009-05-10 12:00:00|2025-03-06 09:35:...|
|      6|     2009|    6|         6|   Monaco Gr

In [0]:
constructors_df_silver = spark.read.format("delta").load(delta_path + "/constructors")

constructors_df_silver_transformed = constructors_df_silver.withColumnRenamed("constructorId","constructor_id") \
           .withColumnRenamed("constructorRef","constructor_ref") \
           .withColumn("ingestion_date", current_timestamp()) \
               .drop("url")

constructors_df_silver_transformed.show()

constructors_df_silver_transformed.write.format("parquet").mode("overwrite").save(silver_path + "/constructors")
               

+--------------+---------------+-----------+-----------+--------------------+
|constructor_id|constructor_ref|       name|nationality|      ingestion_date|
+--------------+---------------+-----------+-----------+--------------------+
|             1|        mclaren|    McLaren|    British|2025-03-06 09:35:...|
|             2|     bmw_sauber| BMW Sauber|     German|2025-03-06 09:35:...|
|             3|       williams|   Williams|    British|2025-03-06 09:35:...|
|             4|        renault|    Renault|     French|2025-03-06 09:35:...|
|             5|     toro_rosso| Toro Rosso|    Italian|2025-03-06 09:35:...|
|             6|        ferrari|    Ferrari|    Italian|2025-03-06 09:35:...|
|             7|         toyota|     Toyota|   Japanese|2025-03-06 09:35:...|
|             8|    super_aguri|Super Aguri|   Japanese|2025-03-06 09:35:...|
|             9|       red_bull|   Red Bull|   Austrian|2025-03-06 09:35:...|
|            10|    force_india|Force India|     Indian|2025-03-

In [0]:
drivers_df_silver = spark.read.format("delta").load(delta_path + "/drivers")

#drivers_df_silver.show()

drivers_df_silver_transformed = drivers_df_silver.withColumnRenamed("driverId","driver_id") \
           .withColumnRenamed("driverRef","driver_ref") \
               .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname"))) \
                   .withColumn("ingestion_date", current_timestamp()) \
                   .drop("name.forename", "name.surname", "url")

drivers_df_silver_transformed.show()

drivers_df_silver_transformed.write.format("parquet").mode("overwrite").save(silver_path + "/drivers")

+----+----------+---------+----------+------------------+-----------+------+--------------------+
|code|       dob|driver_id|driver_ref|              name|nationality|number|      ingestion_date|
+----+----------+---------+----------+------------------+-----------+------+--------------------+
| HAM|1985-01-07|        1|  hamilton|    Lewis Hamilton|    British|    44|2025-03-06 09:42:...|
| HEI|1977-05-10|        2|  heidfeld|     Nick Heidfeld|     German|    \N|2025-03-06 09:42:...|
| ROS|1985-06-27|        3|   rosberg|      Nico Rosberg|     German|     6|2025-03-06 09:42:...|
| ALO|1981-07-29|        4|    alonso|   Fernando Alonso|    Spanish|    14|2025-03-06 09:42:...|
| KOV|1981-10-19|        5|kovalainen| Heikki Kovalainen|    Finnish|    \N|2025-03-06 09:42:...|
| NAK|1985-01-11|        6|  nakajima|   Kazuki Nakajima|   Japanese|    \N|2025-03-06 09:42:...|
| BOU|1979-02-28|        7|  bourdais|Sébastien Bourdais|     French|    \N|2025-03-06 09:42:...|
| RAI|1979-10-17|   

In [0]:
results_df_silver = spark.read.format("delta").load(delta_path + "/results")

results_df_silver_transformed = results_df_silver.withColumnRenamed("resultId","result_id") \
               .withColumnRenamed("raceId","race_id") \
                    .withColumnRenamed("driverId","driver_id") \
                        .withColumnRenamed("constructorId","constructor_id") \
                             .withColumnRenamed("position","position_text") \
                                  .withColumnRenamed("positionOrder","position_order") \
                                       .withColumnRenamed("fastestLap","fastest_lap") \
                                            .withColumnRenamed("fastestLapTime","fastest_lap_time") \
                                                .withColumnRenamed("fastestLapSpeed","fastest_lap_speed") \
                                                    .withColumn("ingestion_date", current_timestamp()) \
                                                       .drop("statusId")

results_df_silver_transformed.show()

results_df_silver_transformed.write.partitionBy("race_id").format("parquet").mode("overwrite").save(silver_path + "/results")

+--------------+---------+-----------+-----------------+----------------+----+----+------------+------+------+-------------+--------------+------------+-------+----+---------+-----------+--------------------+
|constructor_id|driver_id|fastest_lap|fastest_lap_speed|fastest_lap_time|grid|laps|milliseconds|number|points|position_text|position_order|positionText|race_id|rank|result_id|       time|      ingestion_date|
+--------------+---------+-----------+-----------------+----------------+----+----+------------+------+------+-------------+--------------+------------+-------+----+---------+-----------+--------------------+
|             1|        1|         39|            218.3|        1:27.452|   1|  58|     5690616|    22|  10.0|            1|             1|           1|     18|   2|        1|1:34:50.616|2025-03-06 09:48:...|
|             2|        2|         41|          217.586|        1:27.739|   5|  58|     5696094|     3|   8.0|            2|             2|           2|     18|   3

In [0]:
pitstops_df_silver = spark.read.format("delta").load(delta_path + "/pitstops")

pitstops_df_silver_transformed = pitstops_df_silver.withColumnRenamed("pitstopId","pitstop_id") \
               .withColumnRenamed("raceId","race_id") \
                   .withColumn("ingestion_date", current_timestamp())

pitstops_df_silver_transformed.show()

pitstops_df_silver_transformed.write.format("parquet").mode("overwrite").save(silver_path + "/pitstops")

+--------+--------+---+------------+-------+----+--------+--------------------+
|driverId|duration|lap|milliseconds|race_id|stop|    time|      ingestion_date|
+--------+--------+---+------------+-------+----+--------+--------------------+
|     153|  26.898|  1|       26898|    841|   1|17:05:23|2025-03-06 09:59:...|
|      30|  25.021|  1|       25021|    841|   1|17:05:52|2025-03-06 09:59:...|
|      17|  23.426| 11|       23426|    841|   1|17:20:48|2025-03-06 09:59:...|
|       4|  23.251| 12|       23251|    841|   1|17:22:34|2025-03-06 09:59:...|
|      13|  23.842| 13|       23842|    841|   1|17:24:10|2025-03-06 09:59:...|
|      22|  23.643| 13|       23643|    841|   1|17:24:29|2025-03-06 09:59:...|
|      20|  22.603| 14|       22603|    841|   1|17:25:17|2025-03-06 09:59:...|
|     814|  24.863| 14|       24863|    841|   1|17:26:03|2025-03-06 09:59:...|
|     816|  25.259| 14|       25259|    841|   1|17:26:50|2025-03-06 09:59:...|
|      67|  25.342| 15|       25342|    

In [0]:
laptimes_df_silver = spark.read.format("delta").load(delta_path + "/laptimes")

laptimes_df_silver_transformed = laptimes_df_silver.withColumnRenamed("raceId","race_id") \
               .withColumnRenamed("driverId","driver_id") \
                   .withColumn("ingestion_date", current_timestamp())

laptimes_df_silver_transformed.show()

laptimes_df_silver_transformed.write.format("parquet").mode("overwrite").save(silver_path + "/laptimes")

+-------+---------+---+--------+--------+------------+--------------------+
|race_id|driver_id|lap|position|    time|milliseconds|      ingestion_date|
+-------+---------+---+--------+--------+------------+--------------------+
|    841|       20|  1|       1|1:38.109|       98109|2025-03-06 10:00:...|
|    841|       20|  2|       1|1:33.006|       93006|2025-03-06 10:00:...|
|    841|       20|  3|       1|1:32.713|       92713|2025-03-06 10:00:...|
|    841|       20|  4|       1|1:32.803|       92803|2025-03-06 10:00:...|
|    841|       20|  5|       1|1:32.342|       92342|2025-03-06 10:00:...|
|    841|       20|  6|       1|1:32.605|       92605|2025-03-06 10:00:...|
|    841|       20|  7|       1|1:32.502|       92502|2025-03-06 10:00:...|
|    841|       20|  8|       1|1:32.537|       92537|2025-03-06 10:00:...|
|    841|       20|  9|       1|1:33.240|       93240|2025-03-06 10:00:...|
|    841|       20| 10|       1|1:32.572|       92572|2025-03-06 10:00:...|
|    841|   

In [0]:
qualifying_df_silver = spark.read.format("delta").load(delta_path + "/qualifying")

qualifying_df_silver_transformed = qualifying_df_silver.withColumnRenamed("qualifyId","qualify_id") \
               .withColumnRenamed("raceId","race_id") \
                       .withColumnRenamed("driverId","driver_id") \
                            .withColumnRenamed("constructorId","constructor_id") \
                                .withColumn("ingestion_date", current_timestamp())

qualifying_df_silver_transformed.show()

qualifying_df_silver_transformed.write.format("parquet").mode("overwrite").save(silver_path + "/qualifying")

+--------------+---------+------+--------+--------+--------+--------+----------+-------+--------------------+
|constructor_id|driver_id|number|position|      q1|      q2|      q3|qualify_id|race_id|      ingestion_date|
+--------------+---------+------+--------+--------+--------+--------+----------+-------+--------------------+
|             1|        1|    22|       1|1:26.572|1:25.187|1:26.714|         1|     18|2025-03-06 10:02:...|
|             2|        9|     4|       2|1:26.103|1:25.315|1:26.869|         2|     18|2025-03-06 10:02:...|
|             1|        5|    23|       3|1:25.664|1:25.452|1:27.079|         3|     18|2025-03-06 10:02:...|
|             6|       13|     2|       4|1:25.994|1:25.691|1:27.178|         4|     18|2025-03-06 10:02:...|
|             2|        2|     3|       5|1:25.960|1:25.518|1:27.236|         5|     18|2025-03-06 10:02:...|
|             7|       15|    11|       6|1:26.427|1:26.101|1:28.527|         6|     18|2025-03-06 10:02:...|
|         